### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** 27 апреля 2020, 08:30   
**Штраф за опоздание:** -2 балла после 08:30 27 апреля, -4 балла после 08:30 4 мая, -6 баллов после 08:30 11 мая, -8 баллов после 08:30 18 мая.

При отправлении ДЗ указывайте фамилию в названии файла Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:
[ML0220, Задание 3] Фамилия Имя. 


Используйте данный Ipython Notebook при оформлении домашнего задания.

##  Реализуем дерево решений (3 балла)

Допишите недостающие части дерева решений. Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn.
Внимание: если Вас не устраивает предложенная структура хранения дерева, Вы без потери баллов можете сделать свой класс DecisionTreeClassifier, в котором сами полностью воспроизведете алгоритм дерева решений. Обязательно в нем иметь только функции fit, predict

In [282]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

In [446]:
class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=5, criterion='gini'):
        """
        criterion -- критерий расщепления. необходимо релизовать три:
        Ошибка классификации, Индекс Джини, Энтропийный критерий
        max_depth -- максимальная глубина дерева
        min_samples_split -- минимальное число объектов в листе, чтобы сделать новый сплит
        """
        self.criterion = criterion
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.num_class = -1
        # Для последнего задания
        self.feature_importances_ = None
        self.criterion = criterion

        # Структура, которая описывает дерево
        # Представляет словарь, где для  node_id (айдишник узла дерева) храним
        # (тип_узла, айдишник признака сплита, порог сплита) если тип NON_LEAF_TYPE
        # (тип_узла, предсказание класса, вероятность класса) если тип LEAF_TYPE
        # Подразумевается, что у каждого node_id в дереве слева 
        # узел с айди 2 * node_id + 1, а справа 2 * node_id + 2
                
        self.tree = {0: [self.__class__.NON_LEAF_TYPE, -1, 0.0]}
        

    def __div_samples(self, x, y, feature_id, threshold):
        """
        Разделяет объекты на 2 множества
        x -- матрица объектов
        y -- вектор ответов
        feature_id -- айдишник признака, по которому делаем сплит
        threshold -- порог, по которому делаем сплит
        """
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]
    

    def __info(self, l_c, l_s, r_c, r_s, criterion):
        """
        Считаем меру неопределённости
        посредством критерия criterion
        по выборке (x, y)
        """

        if   criterion == 'error':
            return self.__misclass(l_c, l_s, r_c, r_s)
        
        elif criterion == 'gini':
            return self.__gini(l_c, l_s, r_c, r_s)
        
        elif criterion == 'entropy':
            return self.__entropy(l_c, l_s, r_c, r_s)
        
        else:
            raise RuntimeError("No such criterion as \'{}\'!".format(criterion))

            
            
    def __gini(self, l_c, l_s, r_c, r_s):#работаю с матрицами NxKxF. N - число объектов
        l_s = l_s.astype('float')  # K - число классов, F - чилсо признаков. r_c - общее количество 
        r_s = r_s.astype('float')  # объектов справа, r_s количество объектов справа по классам.
                                   # l_s такая матрица, что l_s - r_s = количество объектов слева по классам
                                   # l_c такая матрица, что l_c - r_c = общее количество объектов слева
        return ((r_s-l_s*(r_c/l_c))**2).sum(axis=1)/((l_c-r_c)*r_c).reshape(-1,1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return reduce(lambda ans,x:ans+(x[0]*np.log(x[0]/x[1]+0.000000000001)).sum(axis=1),
                      [[l_s-r_s,l_c-r_c],[r_s,r_c],[-l_s,-l_c]],0)/l_c
    def __misclass(self, l_c, l_s, r_c, r_s):
        return ((l_s-r_s)/(l_c-r_c)).max(axis=1)+(r_s/r_c).max(axis=1)-(l_s/l_c).max(axis=1)

    def __find_threshold(self, x, y):
        y_un=np.unique(y)
        LS=(y[...,np.newaxis]==y_un[np.newaxis]).sum(axis=0)
        if LS.max()==y.shape[0]:
            return [-1,-1]
        
        best_gain=0
        best_feature=best_th=-1
        
        rc=np.arange(1,y.shape[0])
        
        Y=y.shape[0]
        
        feature_ids=np.arange(x.shape[1])
        sorted_ids=x[:,feature_ids].argsort(axis=0)
        rs=(y[sorted_ids][:,np.newaxis]==y_un[np.newaxis,:,np.newaxis]).astype(int)
        for i in range(1,y.shape[0]):
                rs[i]+=rs[i-1]
        R=rs[-1]
        rs=rs[:-1]
        gain=self.__info(Y,R[np.newaxis],rc[...,np.newaxis,np.newaxis],rs, self.criterion)
        r=np.take_along_axis(x,sorted_ids,axis=0)
        gain*=r[:-1]!=r[1:]
        best_idx=gain.argmax()
        if gain.ravel()[best_idx]>0:
            best_feature=feature_ids[best_idx%gain.shape[1]]
            best_th=x[sorted_ids.ravel()[best_idx],best_feature]
            # self.feature_importances_[best_feature]+=y.shape[0]/self.num_samples*gain.ravel()[best_idx]
    
        return best_feature,best_th
        

    def __fit_node(self, x, y, node_id, depth):
        """
        Делаем новый узел в дереве
        Решаем, терминальный он или нет
        Если нет, то строим левый узел  с айди 2 * node_id + 1
        И правый узел с  айди 2 * node_id + 2
        """
        # if trunc(np.log2(node_id+1)) > depth:
        #      return
    
        c_node = self.tree[node_id]
        
        y_len = y.shape[0]
        
        prob_distr = np.bincount(y, minlength=self.num_class,\
                                    weights=[1/y_len]*y_len)
        
        condition = (x.shape[0] <= self.min_samples_split) or (np.isclose(np.max(prob_distr), 1.0))  
        
        if condition:
            c_node[0] = self.__class__.LEAF_TYPE        
            c_node[1] = np.argmax(prob_distr)
            c_node[2] = prob_distr[c_node[1]]
        
        else:
            c_node[1], c_node[2] = self.__find_threshold(x, y)
            x_left, x_right, y_left, y_right = self.__div_samples(x, y, c_node[1], c_node[2])
            c_node[0] = self.__class__.NON_LEAF_TYPE
            self.tree[2*node_id+1] = [None, None, None]
            self.tree[2*node_id+2] = [None, None, None]  
            self.__fit_node(x_left,  y_left,  2*node_id+1, depth+1)
            self.__fit_node(x_right, y_right, 2*node_id+2, depth+1)

    
    def fit(self, x, y):        
        """
        Рекурсивно строим дерево решений
        Начинаем с корня node_id 0
        """
        self.num_class = np.unique(y).size
        self.__fit_node(np.asarray(x), np.asarray(y), 0, 0)
            

    def __predict_class(self, x, node_id):
        """
        Рекурсивно обходим дерево по всем узлам,
        пока не дойдем до терминального
        """
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]
        
        
    def predict(self, X):
        """
        Вызывает predict для всех объектов из матрицы X
        """
        return np.array([self.__predict_class(x, 0) for x in np.asarray(X)])

    
    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

    
    def get_feature_importance():
        """
        Возвращает важность признаков
        """
        # Ваш код здесь
        pass

In [447]:
a = np.array([1,2,3])
b = np.array([10,20])
tmp1 = np.repeat(a, b.shape[0])
tmp2 = np.tile(b, a.shape[0])
np.dstack((tmp1,tmp2))[0]

array([[ 1, 10],
       [ 1, 20],
       [ 2, 10],
       [ 2, 20],
       [ 3, 10],
       [ 3, 20]])

In [448]:
# TEST:

x = np.array([[0,7, 0.01],[1.2,-5,-0.03],[0.6,7,0.09],[-1,-5,0.07],[-0.4,4,-0.05],[-1.2,9,0.04]])
y = np.array([0,2,0,2,2,1])

x, y

(array([[ 0.  ,  7.  ,  0.01],
        [ 1.2 , -5.  , -0.03],
        [ 0.6 ,  7.  ,  0.09],
        [-1.  , -5.  ,  0.07],
        [-0.4 ,  4.  , -0.05],
        [-1.2 ,  9.  ,  0.04]]), array([0, 2, 0, 2, 2, 1]))

In [449]:
y_un = np.unique(y)
LS = (y[...,np.newaxis] == y_un[np.newaxis]).sum(axis=0)
if LS.max()==y.shape[0]:
    print("blaaa")
    
best_gain=0
best_feature=best_th=-1

rc=np.arange(1,y.shape[0])

Y=y.shape[0]

feature_ids=[0,2]
sorted_ids=x[:,feature_ids].argsort(axis=0)

# print(y[sorted_ids][:,np.newaxis].shape, y_un[np.newaxis,:,np.newaxis].shape, sep='\n\n')
# y[sorted_ids][:,np.newaxis]==y_un[np.newaxis,:,np.newaxis]
rs=(y[sorted_ids][:,np.newaxis]==y_un[np.newaxis,:,np.newaxis]).astype(int)
for i in range(1,y.shape[0]):
        rs[i]+=rs[i-1]
R=rs[-1]
rs=rs[:-1]
R

array([[2, 2],
       [1, 1],
       [3, 3]])

In [450]:
# END.

In [451]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2, criterion='gini')
clf = DecisionTreeClassifier(min_samples_split=2, criterion='gini')

In [452]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

In [453]:
my_clf.fit(X_train, y_train)
clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [454]:
accuracy_score(y_pred=clf.predict(X_test), y_true=y_test), accuracy_score(y_pred=my_clf.predict(X_test), y_true=y_test)

(0.8888888888888888, 0.8888888888888888)

In [455]:
from IPython.display import Image  
from sklearn import tree
import pydotplus # installing pyparsing maybe needed

dot_data = tree.export_graphviz(clf, out_file=None, feature_names = None)
graph = pydotplus.graph_from_dot_data(dot_data)
#Image(graph.create_png())

In [456]:
Image(graph.create_png())

InvocationException: GraphViz's executables not found

In [457]:
my_clf.tree

{0: [0, 9, 3.8],
 1: [0, 6, 1.57],
 2: [0, 12, 985.0],
 3: [0, 12, 714.0],
 4: [0, 2, 2.02],
 7: [1, 0, 1.0000000000000004],
 8: [0, 0, 13.11],
 17: [1, 0, 1.0],
 18: [1, 1, 1.0],
 9: [1, 2, 1.0000000000000007],
 10: [1, 1, 1.0],
 5: [1, 0, 1.0],
 6: [1, 1, 1.0000000000000002]}

## Ускоряем дерево решений (2 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine. 
Для этого используем numpy.

In [458]:
%timeit clf.fit(X_train, y_train)

725 µs ± 3.83 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [459]:
%timeit my_clf.fit(X_train, y_train)

3.99 ms ± 35.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Боевое применение (3 балла)

На практике Вы познакомились с датасетом Speed Dating Data. В нем каждая пара в быстрых свиданиях характеризуется определенным набором признаков. Задача -- предсказать, произойдет ли матч пары (колонка match). 

Пример работы с датасетом можете найти в практике пункт 2
https://github.com/VVVikulin/ml1.sphere/blob/master/2019-09/lecture_06/pract-trees.ipynb

Данные и описания колонок лежат тут
https://cloud.mail.ru/public/8nHV/p6J7wY1y1/speed-dating-experiment/

Скачайте датасет, обработайте данные, как показано на семинаре или своим собственным способом. Обучите дерево классифкации. В качестве таргета возьмите колонку 'match'. Постарайтесь хорошо обработать признаки, чтобы выбить максимальную точность. Если точность будет близка к случайному гаданию, задание не будет защитано. 


In [497]:
df = pd.read_csv('Speed Dating Data.csv', encoding = "ISO-8859-1")
pd.options.display.max_columns = 200

In [498]:
df.isnull().sum(axis=0)

iid            0
id             1
gender         0
idg            0
condtn         0
            ... 
attr5_3     6362
sinc5_3     6362
intel5_3    6362
fun5_3      6362
amb5_3      6362
Length: 195, dtype: int64

In [499]:
df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,partner,pid,match,int_corr,samerace,age_o,race_o,pf_o_att,pf_o_sin,pf_o_int,pf_o_fun,pf_o_amb,pf_o_sha,dec_o,attr_o,sinc_o,intel_o,fun_o,amb_o,shar_o,like_o,prob_o,met_o,age,field,field_cd,undergra,mn_sat,tuition,race,imprace,imprelig,from,zipcode,income,goal,date,go_out,career,career_c,sports,tvsports,exercise,dining,museums,art,hiking,gaming,clubbing,reading,tv,theater,movies,concerts,music,shopping,yoga,exphappy,expnum,attr1_1,sinc1_1,intel1_1,fun1_1,amb1_1,shar1_1,attr4_1,sinc4_1,intel4_1,fun4_1,amb4_1,shar4_1,attr2_1,sinc2_1,intel2_1,fun2_1,amb2_1,shar2_1,attr3_1,sinc3_1,fun3_1,intel3_1,amb3_1,attr5_1,sinc5_1,intel5_1,fun5_1,amb5_1,dec,attr,sinc,intel,fun,amb,shar,like,prob,met,match_es,attr1_s,sinc1_s,intel1_s,fun1_s,amb1_s,shar1_s,attr3_s,sinc3_s,intel3_s,fun3_s,amb3_s,satis_2,length,numdat_2,attr7_2,sinc7_2,intel7_2,fun7_2,amb7_2,shar7_2,attr1_2,sinc1_2,intel1_2,fun1_2,amb1_2,shar1_2,attr4_2,sinc4_2,intel4_2,fun4_2,amb4_2,shar4_2,attr2_2,sinc2_2,intel2_2,fun2_2,amb2_2,shar2_2,attr3_2,sinc3_2,intel3_2,fun3_2,amb3_2,attr5_2,sinc5_2,intel5_2,fun5_2,amb5_2,you_call,them_cal,date_3,numdat_3,num_in_3,attr1_3,sinc1_3,intel1_3,fun1_3,amb1_3,shar1_3,attr7_3,sinc7_3,intel7_3,fun7_3,amb7_3,shar7_3,attr4_3,sinc4_3,intel4_3,fun4_3,amb4_3,shar4_3,attr2_3,sinc2_3,intel2_3,fun2_3,amb2_3,shar2_3,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,1,11.0,0,0.14,0,27.0,2.0,35.0,20.0,20.0,20.0,0.0,5.0,0,6.0,8.0,8.0,8.0,8.0,6.0,7.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,6.0,9.0,7.0,7.0,6.0,5.0,7.0,6.0,2.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,2,12.0,0,0.54,0,22.0,2.0,60.0,0.0,0.0,40.0,0.0,0.0,0,7.0,8.0,10.0,7.0,7.0,5.0,8.0,4.0,2.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,7.0,8.0,7.0,8.0,5.0,6.0,7.0,5.0,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,3,13.0,1,0.16,1,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,1,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,1.0,21.0,Law,1.0,NaN,NaN,NaN,4.0,2.0,4.0,Chicago,"60,521","69,487.00",2.0,7.0,1.0,lawyer,NaN,9.0,2.0,8.0,9.0,1.0,1.0,5.0,1.0,5.0,6.0,9.0,1.0,10.0,10.0,9.0,8.0,1.0,3.0,2.0,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,35.0,20.0,15.0,20.0,5.0,5.0,6.0,8.0,8.0,8.0,7.0,NaN,NaN,NaN,NaN,NaN,1,5.0,8.0,9.0,8.0,5.0,7.0,7.0,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,19.44,16.67,13.89,22.22,11.11,16.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,7.0,8.0,7.0,6.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,15.0,20.0,20.0,15.0,15.0,15.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,4,14.0,1,0.61,0,23.0,2.0,30.0,5.0,15.0,40.0,5.0,5.0,1,7.0,8.0,9.0,8.0,9

Разбейте датасет на трейн и валидацию. Подберите на валидации оптимальный критерий  информативности. 
Постройте графики зависимости точности на валидации от глубины дерева, от минимального числа объектов для сплита. 
Какой максимальной точности удалось достигнуть?

## Находим самые важные признаки (2 балла)



По построенному дереву  легко понять, какие признаки лучше всего помогли решить задачу. Часто это бывает нужно  не только  для сокращения размерности в данных, но и для лучшего понимания прикладной задачи. Например, Вы хотите понять, какие признаки стоит еще конструировать -- для этого нужно понимать, какие из текущих лучше всего работают в дереве. 

Самый простой метод -- посчитать число сплитов, где использовался данные признак. Это не лучший вариант, так как по признаку который принимает всего 2 значения, но который почти точно разделяет выборку, число сплитов будет очень 1, но при этом признак сам очень хороший. 
В этом задании предлагается для каждого признака считать суммарный gain (в лекции обозначено как Q) при использовании этого признака в сплите. Тогда даже у очень хороших признаков с маленьким число сплитов это значение должно быть довольно высоким.  

Реализовать это довольно просто: создаете словарь номер фичи : суммарный гейн и добавляете в нужную фичу каждый раз, когда используете ее при построении дерева. 

Добавьте функционал, который определяет значения feature importance. Обучите дерево на датасете Speed Dating Data.
Выведите 10 главных фичей по важности.

## Фидбек (бесценно)

* Какие аспекты обучения деревьев решений Вам показались непонятными? Какое место стоит дополнительно объяснить?

### Ваш ответ здесь

* Здесь Вы можете оставить отзыв о этой домашней работе или о всем курсе.

### ВАШ ОТЗЫВ ЗДЕСЬ

